In [18]:
import torch
from datasets import load_dataset
from transformers import (
    RobertaTokenizerFast,
    RobertaForSequenceClassification,
    TrainingArguments,
    Trainer,
    AutoConfig,
)
# from sklearnex import patch_sklearn
# patch_sklearn()

import json
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
import pickle
import numpy as np
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

import torch
from torch import nn
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split
from transformers import BertForSequenceClassification
import transformers
from torch.utils.data import Dataset, DataLoader
#from transformers import *
import torch.optim as optim
# from tqdm import tqdm
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import scipy
from scipy.stats import pearsonr
from torch.optim.lr_scheduler import StepLR

torch.manual_seed(0)




In [19]:
PATH='train_file.json'
EOS='</s>'
SEP='[SEP]'
SOS='<s>'
BATCH_SIZE=16
NUM_EPOCHS=10
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_id = "roberta-large"
label_encoder = LabelEncoder()
tokenizer=AutoTokenizer.from_pretrained('bert-base-uncased')

# dataset

In [20]:


# read json
with open(PATH) as f:
    data = json.load(f)
    # to pandas
    df = pd.DataFrame(data)


In [21]:
df.head()

,episode,speakers,emotions,utterances,triggers
0,utterance_3492,"[Phoebe, Eric, Phoebe, Eric, Phoebe]","[surprise, fear, surprise, sadness, disgust]","[You-you you had sex with Ursula?!, Uh, a litt...","[1.0, 1.0, 0.0, 0.0, 0.0]"
1,utterance_3952,"[Monica, Monica, Phoebe, Joey, Joey, Joey, Rac...","[disgust, disgust, anger, sadness, surprise, a...","[Dad, please don't pick your teeth out here!, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,utterance_3198,"[Older Scientist, Ross, Ross, Joey, Ross, Ross...","[neutral, neutral, neutral, neutral, neutral, ...","[Dr. Geller, there's a seat over here., Thank ...","[0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0]"
3,utterance_2834,"[Monica, Monica, Monica]","[neutral, surprise, neutral]","[So, how'd the lasagne go over?, Really?!, Good.]","[0.0, 0.0, 1.0]"
4,utterance_453,"[Kate, The Director, Kate]","[joy, sadness, sadness]","[Become a drama critic!, I am hurt! A plague ...","[0.0, 0.0, 1.0]"


In [22]:
sentence_len = []
text=[]


for i in range(len(df)):
    uterances=df.iloc[i]['utterances']
    speaker=df.iloc[i]['speakers']
    length=0
    sentence=SOS+' '
    for (speaker,uterance) in zip(speaker,uterances):
        sentence+=speaker+':'+uterance+' '+SEP+' '

        length+=len(uterance.split())
    if(length==2):
        print(sentence)
        print(df.iloc[i])
    sentence_len.append(length)
    text.append(sentence+EOS)
    # print(length)
    # break

<s> Phoebe:No. [SEP] Phoebe:No! [SEP] 
episode          utterance_915
speakers      [Phoebe, Phoebe]
emotions      [neutral, anger]
utterances          [No., No!]
triggers            [0.0, 0.0]
Name: 968, dtype: object
<s> Phoebe:No. [SEP] Phoebe:No! [SEP] 
episode          utterance_915
speakers      [Phoebe, Phoebe]
emotions      [neutral, anger]
utterances          [No., No!]
triggers            [0.0, 0.0]
Name: 3984, dtype: object


In [23]:
# config =AutoConfig.from_pretrained(model_id)
# config.update({"id2label": label_to_id})
# tokenizer = RobertaTokenizerFast.from_pretrained(model_id)
# model = RobertaForSequenceClassification.from_pretrained(model_id,config=config)

In [24]:
text_df=pd.DataFrame(text,columns=['text'])
print(len(text),len(set(text)))
text_df.head()

6740 4179


,text
0,<s> Phoebe:You-you you had sex with Ursula?! [...
1,"<s> Monica:Dad, please don't pick your teeth o..."
2,"<s> Older Scientist:Dr. Geller, there's a seat..."
3,"<s> Monica:So, how'd the lasagne go over? [SEP..."
4,<s> Kate:Become a drama critic! [SEP] The Dire...


In [25]:


label_encoding ={
    'neutral':0,
    'surprise':1,
    'sadness':2,
    'anger':3,
    'fear':4,
    'disgust':5,
    'joy':6
}



In [26]:


y_train=[]
for i in df["emotions"]:
    lst=[]
    for j in i:
        lst.append(label_encoding[j])
    
    lst.extend([128]*(7-len(lst)))
    y_train.append(lst)

x_train=text_df['text']


In [27]:
class CustomDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data=data
        self.tokenizer = tokenizer
        
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x = self.data.iloc[idx, 0]
        y = self.data.iloc[idx, 1]
        encoded_x = self.tokenizer(x,
                                      add_special_tokens=True, 
                                      padding='max_length', 
                                      truncation=True, 
                                      return_tensors='pt')
        input_ids = encoded_x['input_ids'].squeeze(0)
        attention_mask = encoded_x['attention_mask'].squeeze(0)
        return input_ids,attention_mask,torch.tensor(y)

In [28]:
train_dataframe=pd.DataFrame({'text':x_train,'label':y_train})
train_data=CustomDataset(train_dataframe,tokenizer)
train_dataloader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
# val_dataframe=pd.DataFrame({'text':x_val,'label':y_val})

# model

In [29]:
torch.cuda.empty_cache()

In [30]:
def plotgraph(train_losses,val_losses):
    x=[i+1 for i in range(NUM_EPOCHS)]
    plt.plot(train_losses, label='Training Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss per Epoch')
    plt.legend()
    plt.savefig('loss_graph.png')
    plt.show()

In [31]:
model=BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=3)
# model.classifier = nn.Sequential(
#     nn.Linear(model.config.hidden_size, 1),
#     nn.Sigmoid()  # Output float between 0 and 1
# )
model.to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)
# optimizer=torch.optim.SGD(model.parameters(),lr=0.0005,weight_decay=0.001)
loss_fn=nn.CrossEntropyLoss()

model.safetensors: 100%|██████████| 440M/440M [00:20<00:00, 21.7MB/s] 
c:\Users\acer\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\acer\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of BertForS

In [32]:
import gc
import sys
gc.collect()
torch.cuda.empty_cache()

train_losses=[]
val_losses=[]